In [1]:
import os
import azureml.core
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core.environment import Environment
from azureml.core import ScriptRunConfig
from azureml.core import Dataset
from azureml.data.datapath import DataPath
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core.conda_dependencies import CondaDependencies

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.39.0


### Prihlasenie do azure

In [2]:
ws = Workspace.from_config('azure_config.json', auth=InteractiveLoginAuthentication())
print(ws.name, ws.location, ws.resource_group, sep='\t')

a0047stufiitml01	westeurope	a0047-STUFIIT-ML01


### Nacitanie datasetu

In [3]:
dataset = Dataset.get_by_name(workspace=ws, name='BraTS-dataset')

In [4]:
# nefunguje :(
#dataset.to_path()[0]

## Experimenty

In [15]:
config = ScriptRunConfig(source_directory='../src/',
                         script='train.py',
                         arguments=['--wandb', "a68b79c89593d1f8c50631508f2e618f3ba7ccf3",
                                    '--data_path', dataset.as_mount()], # This is important how to mount dataset from DataStore
                         compute_target='P100x1-xpovinec') # Compute target is your created compute cluster


In [16]:
experiment = Experiment(workspace=ws, name='xpovinec-test')

### Nastavenie env

In [ ]:
# toto vypise vsetky env
for i,key in enumerate(ws.environments.keys()):
    print(f"{i}.",key)

In [ ]:
ws.environments

In [19]:
# vytvorenie noveho env
conda = CondaDependencies()

# add channels
#conda.add_channel('pytorch')

# add conda packages
conda.add_conda_package('pip=20.2.4')
conda.add_conda_package('python=3.8')

# add pip packages
conda.add_pip_package('keras~=2.6.0')
#conda.add_pip_package('Keras-Preprocessing~=2.8.0')
conda.add_pip_package('matplotlib~=3.5.1')
conda.add_pip_package('nibabel~=3.2.2')
conda.add_pip_package('numpy~=1.21.3')
conda.add_pip_package('pandas~=1.4.1')
conda.add_pip_package('scikit-learn~=1.0.2')
conda.add_pip_package('scipy~=1.7.0')
conda.add_pip_package('tensorflow-gpu~=2.7.0')
conda.add_pip_package('azureml-core~=1.39.0')
conda.add_pip_package('opencv-python~=4.5.5.64')
conda.add_pip_package('opencv-python-headless~=4.5.5.64')
conda.add_pip_package('scikit-image')
conda.add_pip_package('segmentation-models-3D')

# create environment
env = Environment('xpovinec-env')
env.python.conda_dependencies = conda
env.save_to_directory('../', overwrite=True)

In [ ]:
env

In [62]:
# vytvorenie noveho env z requirements.txt
#env = Environment.from_pip_requirements(name='xpovinec-env', file_path='../requirements.txt')

In [ ]:
# pridanie na azure, ked zmenim nejaky package tak sa updatne
env.register(workspace=ws)

In [18]:
# nieco s dockerom, neviem ci to treba davat vzdy
azureml.core.runconfig.DockerConfiguration(use_docker=True)

In [17]:
# vyber env
env = Environment.get(workspace=ws, name='xpovinec-env')
#env

In [18]:
# nastavenie env pre ten experiment
config.run_config.environment = env

### Spustenie experimentu

In [19]:
run = experiment.submit(config)

Submitting /Users/kuko/Developer/BP/src directory for run. The size of the directory >= 25 MB, so it can take a few minutes.


In [10]:
# toto zisti, kde bezi a vrati to stranku
aml_url = run.get_portal_url()
print(aml_url)

https://ml.azure.com/runs/xpovinec-test_1651557556_0e831090?wsid=/subscriptions/bc82dccd-f19d-42cb-9ce3-0d5df33ef086/resourcegroups/a0047-STUFIIT-ML01/workspaces/a0047stufiitml01&tid=5dbf1add-202a-4b8d-815b-bf0fb024e033
